# Quickstart

This guide demonstrates how to generate interactive forms from `JSON Schema`
 or `pydantic` models using `ipywidgets-jsonschema`.

## Basic example with `JSON schema`
`ipywidgets-jsonschema` allows you to generate a widget form from an existing schema that follows the JSON Schema specification. We first define a schema:

In [28]:
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "title": "Name"},
        "food": {
            "type": "array",
            "items": {"type": "string"},
            "title": "Preferred Food",
        },
    },
}

Generating the widget form for it and visualizing it in Jupyter is as simple as this:

In [29]:
from ipywidgets_jsonschema import Form

form = Form(schema)
form.show()

This will display a form in Jupyter where users can input data according to the schema.
To access current data, we can use the `data` property of `form`:

In [30]:
form.data

{'name': '', 'food': []}

This will automatically validate the data.

## Basic example using `pydantic`
`ipywidgets-jsonschema`can be easily integrated with `pydantic` models.
First create a `pydantic` model:

In [31]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    adress: str

Then create the schema using `model_json_schema()`:

In [32]:
schema_user = User.model_json_schema()
schema_user

{'properties': {'name': {'title': 'Name', 'type': 'string'},
  'adress': {'title': 'Adress', 'type': 'string'}},
 'required': ['name', 'adress'],
 'title': 'User',
 'type': 'object'}

Then generate the widget form as usual:

In [33]:
from ipywidgets_jsonschema import Form

form = Form(schema_user)
form.show()

Alternatively, you can create a widget form directly from the pydantic model without needing to generate a JSON schema manually:

In [34]:
form = Form(User)
form.show()

# Customizing layout and styling

`ipywidgets-jsonschema` provides some customization options to control the appearance and behavior of generated forms. This includes adjusting the layout, enabling sliders for numeric input, preconstructing array items and configuring date/time formats.
Here is an example:

In [35]:
class MyModel(BaseModel):
    name: str
    height: int

In [36]:
form = Form(
    MyModel,
    vertically_place_labels=True,
    use_sliders=True,
)
form.show(width="500px")

See the [Class reference](api/modules.html) for more details.

## Using `PydanticEditorMixin` for interactive pydantic model editing

`PydanticEditorMixin` allows interactive editing of pydantic models in Jupyter. It extends any model, allowing you to edit its attributes dynamically using an interactive form. It provides: 
* A Form Interface: Displays the model’s attributes as editable fields.
* Edit Mode Toggle: Enter edit mode and change values.
* Save & Cancel Buttons: Save changes or revert to the original values.


First import `PydanticEditorMixin` using:

In [37]:
from ipywidgets_jsonschema.pydantic_editor_mixin import PydanticEditorMixin

Then simply inherit from PydanticEditorMixin along with your pydantic model:

In [38]:
class MyModel(PydanticEditorMixin, BaseModel):
    name: str
    age: int
    email: str


model = MyModel(name="John Doe", age=30, email="john.doe@example.com")
model.edit()

This will render a form where each field can be updated, with Cancel or Save buttons to confirm or discard the changes.

# In-Depth Walkthrough and supported Data Types
ipywidgets-jsonschema provides a way to dynamically generate ipywidgets from JSON Schema definitions. This guide covers how different data types are handled and how to use create widgets using JSON schema and pydantic.

## Basic Data Types

The most common types like str, int, and float are rendered as simple input fields. bool is shown as a checkbox.

In [39]:
from ipywidgets_jsonschema import Form
from pydantic import BaseModel, Field


class BasicTypes(BaseModel):
    name: str
    email: str
    age: int
    height: float
    is_active: bool


form = Form(BasicTypes)
form.show()

## Default Values
Setting the default values is simple as:

In [40]:
class DefaultValues(BaseModel):
    value: int = 10


form = Form(DefaultValues)
form.show()

When using Fields, the default value has to be specified inside the Field:

In [41]:
class DefaultValuesField(BaseModel):
    value: int = Field(default=10)


form = Form(DefaultValues)
form.show()

## In-Depth widget guide
This section covers advanced features such as nested structures, recursion, constraints, enums, and string formats.

## Contraints
JSON Schema constraints such as minLength, maxLength, minimum, maximum, pattern, and format are fully supported. These constraints allow you to have fine-grained control over user input, ensuring that data is validated at the widget level.

To define constraints in a Pydantic model, use the Field class:

In [42]:
from pydantic import Field


class ConstraintsExample(BaseModel):
    number: int = Field(minimum=0, maximum=10)
    text: str = Field(min_length=3, max_length=10, pattern="^[a-zA-Z]+$")


form = Form(ConstraintsExample)
form.show()

## String formats
JSON Schema allows specifying string formats to enforce validation rules on user input. The following formats are supported:

 - email
 - hostname
 - ipv4
 - ipv6
 - uri
 - uuid

To enforce these string formats, it is simple as adding a format key to your JSON schema or pydantic model:


In [43]:
class StringFormats(BaseModel):
    email: str = Field(default="default@example.com", format="email")
    hostname: str = Field(default="default.com", format="hostname")
    ip4address: str = Field(default="127.0.0.1", format="ipv4")
    ip6address: str = Field(default="::1", format="ipv6")
    uri: str = Field(default="http://example.com", format="uri")
    uuid: str = Field(default="00000000-0000-0000-0000-000000000000", format="uuid")


form = Form(StringFormats.model_json_schema())
form.show()

ipywidgets_jsonschema also features visual feedback for user input that doesn't match the specified pattern or format. When users enter invalid data, the corresponding widget provides real-time validation cues, such as highlighting the input field or displaying an error message. 

In [44]:
class Feedback(BaseModel):
    email: str = Field(format="email")
    email_wrong: str = Field(format="email")


form = Form(Feedback)
form.show()

## Datetime

Date and time fields are rendered using special widgets that let you select a date from a calendar or set the time.

In [45]:
from datetime import date, time


class Date(BaseModel):
    date: date
    time: time


form = Form(Date)
form.show()

## List

When your schema includes a List, it will automatically generate a dynamic widget list, that allows users to:
1. Add new entries
2. Reorder list entries
3. Remove individual entries

In [46]:
from typing import List


class Fruits(BaseModel):
    fruit: List[str]


form = Form(Fruits)
form.show()

When using Set in your schema or Pydantic model, the form behaves like a list input but ensures that no duplicate entries are allowed.

In [47]:
from typing import Set


class IDs(BaseModel):
    id: Set[int]


form = Form(IDs)
form.show()

## Nested schema

We fully support **nested schemas**, allowing deeply structured form with embedded classes.

In [48]:
from typing import List


class Address(BaseModel):
    street: str
    city: str


class User(BaseModel):
    name: str
    adresses: List[Address]


form = Form(User)
form.show()

## Enum

When your model includes an Enum field, the form renders a dropdown widget, allowing you to select one of the available options.

In [49]:
from enum import Enum


class ColorEnum(Enum):
    RED = "red"
    BLUE = "blue"
    GREEN = "green"


class Colors(BaseModel):
    color: ColorEnum


form = Form(Colors)
form.show()

## Anyof

We support fields defined using `Union`, which correspond to JSON Schema’s `anyOf`. These allow a field to accept multiple types. The form provides a dropdown, that allows you to select a type.

In [50]:
from typing import Union


class Item(BaseModel):
    price: Union[int, float]


form = Form(Item)
form.show()

## Object / Dictionary

You can use Dict to define dynamic key-value mapping. Keys are strings, and values can be any supported data type.

In [51]:
from typing import Union, Dict


class DictUnion(BaseModel):
    settings: Dict[str, str]


form = Form(DictUnion)
form.show()

## Recursion


Recursive models are supported by using string annotations. This allows a model to reference itself, which is useful for things like nested categories or tree structures.

In [52]:
from typing import List


class Category(BaseModel):
    name: str
    subcategories: List["Category"]


form = Form(Category)
form.show()